In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,AveragePooling2D, Dropout,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
#from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [22]:
os.getcwd()
os.chdir('dataset/Face-Mask-Dataset')
os.getcwd()

'C:\\Users\\Abdul Ahad\\Face-Mask-Detection-COVID19\\dataset\\Face-Mask-Dataset'

In [23]:
INIT_LR = 1e-3
EPOCHS = 4
BS = 64

directory_train = os.getcwd()+r"\Train"
directory_valid = os.getcwd()+r"\Validation"
categories = ["WithMask", "WithoutMask"]

#Grabbing the dataset to train the model from the directory and converting it to numpy array, preprocessing it using mobilenetV2
# preprocessing and categorize the images.
print("[INFO] loading images...")

train_data = []
train_labels = []
valid_data = []
valid_labels = []
for category in categories:
    path = os.path.join(directory_train, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
    	image = preprocess_input(image)
    	train_data.append(image)
    	train_labels.append(category)

# One-Hot Encoding to get 0s and 1s.
lb = LabelBinarizer()
train_labels = lb.fit_transform(labels)
train_labels = to_categorical(labels)

train_data = np.array(data, dtype="float32")
train_labels = np.array(labels)

#Doing the same for validation set now.
for category in categories:
    path = os.path.join(directory_valid, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
    	image = preprocess_input(image)
    	valid_data.append(image)
    	valid_labels.append(category)
        
# One-Hot Encoding to get 0s and 1s.
lb = LabelBinarizer()
valid_labels = lb.fit_transform(valid_labels)
valid_labels = to_categorical(valid_labels)

valid_data = np.array(valid_data, dtype="float32")
valid_labels = np.array(valid_labels)

os.chdir('..//..//')

# (trainX, testX, trainY, testY) = train_test_split(data, labels,
# 	test_size=0.20, stratify=labels, random_state=42)

[INFO] loading images...


In [24]:
train_data, train_labels = shuffle(train_data,train_labels)

In [25]:
print("Training data shape: " + str(data.shape))
print("Validation data shape: " + str(valid_data.shape))
#train_data = np.save('train_data.npy',data)

Training data shape: (10000, 224, 224, 3)
Validation data shape: (800, 224, 224, 3)


## Creating a model 

In [26]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

baseModel = MobileNetV2(weights="imagenet", include_top=False,input_shape=(224,224,3))

# construct the head of the model that will be placed on top of the
# the base model
x = baseModel.output
x = AveragePooling2D(pool_size=(7, 7))(x)
x = Flatten(name="flatten")(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(2, activation="softmax")(x)

model = Model(inputs=baseModel.input, outputs=x)

for layer in baseModel.layers:
	layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=0.0001)
model.compile(
    loss="categorical_crossentropy",
    optimizer=opt,
    metrics=["accuracy"])
model.summary()

[INFO] compiling model...
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_____________________________________________________________

In [ ]:
# Training the model
print("[INFO] training head...")
H = model.fit(
	aug.flow(train_data, train_labels, batch_size=BS),
	steps_per_epoch=len(train_data) // BS,
	validation_data=(valid_data, valid_labels),
	epochs=EPOCHS,
    batch_size=BS,
)


In [46]:
if (not(os.path.exists(os.getcwd()+r'\models\mask-model-2.h5'))):
    model.save('mask-model-2.h5')

## Testing of Model

In [49]:
from tensorflow.keras.models import load_model
loaded_model = load_model(os.getcwd()+r'\models\mask-model-2.h5')

In [48]:
os.chdir('dataset/Face-Mask-Dataset')
directory_test = os.getcwd()+r"\Test"
categories = ["WithMask", "WithoutMask"]

#Grabbing the dataset to train the model from the directory and converting it to numpy array, preprocessing it using mobilenetV2
# preprocessing and categorize the images.
print("[INFO] loading images...")

test_data = []
test_labels = []
for category in categories:
    path = os.path.join(directory_test, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
    	image = preprocess_input(image)
    	test_data.append(image)
    	test_labels.append(category)

# One-Hot Encoding to get 0s and 1s.
lb = LabelBinarizer()
test_labels = lb.fit_transform(labels)
test_labels = to_categorical(labels)

test_data = np.array(data, dtype="float32")
test_labels = np.array(labels)
os.chdir('..//..//')

[INFO] loading images...


In [51]:
loaded_model.evaluate(x=data,y=labels,batch_size=32)

313/313 [==============================] - 257s 821ms/step - loss: 0.0111 - accuracy: 0.9961


[0.011096361093223095, 0.9961000084877014]

## Evaluation Metrics

### In progress....

In [ ]:
#predIdxs = model.predict(testX, batch_size=BS)

In [ ]:
#predIdxs = np.argmax(predIdxs, axis=1)
#predIdxs.shape

In [ ]:
#print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))

In [ ]:
# All the graphs and everything 